In [7]:
import datetime, multiprocessing, logging, os, re, shutil, sys, sqlite3, subprocess, time
import pandas as pd
from utils import *

##################
### Administration
##################
logging.basicConfig(filename= 'log.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
makedb('dbase_Sqlite', 'Malay_Data', "(CHROM int(2), POS int(10), N_ALLELES int(1), N_CHR int(4), ALLELE_FREQ_1 float(7,6), ALLELE_FREQ_2 float(7,6))")
makedb('dbase_Sqlite', 'Malay_rsID', "(CHROM int(2), POS int(10), ID str(15))")
makedb('dbase_Sqlite', 'Indian_Data', "(CHROM int(2), POS int(10), N_ALLELES int(1), N_CHR int(4), ALLELE_FREQ_1 float(7,6), ALLELE_FREQ_2 float(7,6))")
makedb('dbase_Sqlite', 'Indian_rsID', "(CHROM int(2), POS int(10), ID str(15))")
makedb('dbase_Sqlite', 'Chinese_Data', "(CHROM int(2), POS int(10), N_ALLELES int(1), N_CHR int(4), ALLELE_FREQ_1 float(7,6), ALLELE_FREQ_2 float(7,6))")
makedb('dbase_Sqlite', 'Chinese_rsID', "(CHROM int(2), POS int(10), ID str(15))")

In [3]:
workingFolder_Indian = "SgIndian_vcf/dataFreeze_Feb2013/SNP/biAllele/"

workingFolder_Malay = "SgMalay_vcf/2012_05/snps/"

workingFolder_Chinese = "1000G/Phase3/integrated/"

# Filing number of unique samples found in the working folder...

freqFiles_Indian = [f for f in os.listdir(workingFolder_Indian) if re.match(r'chr\d+_analysis_exome\.frq', f)]
rsIDFiles_Indian = [f for f in os.listdir(workingFolder_Indian) if re.match(r'chr\d+_rsID', f)]
freqFiles_Malay = [f for f in os.listdir(workingFolder_Malay) if re.match(r'chr\d+_analysis_exome\.frq', f)]
rsIDFiles_Malay = [f for f in os.listdir(workingFolder_Malay) if re.match(r'chr\d+_rsID', f)]
freqFiles_Chinese = [f for f in os.listdir(workingFolder_Chinese) if re.match(r'chr\d+_analysis_exome\.frq', f)]
rsIDFiles_Chinese = [f for f in os.listdir(workingFolder_Chinese) if re.match(r'chr\d+_rsID', f)]

freqFilesID_pre = re.compile(r'(chr\d+)_analysis_exome\.frq')
freqFilesID = []
for file in freqFiles_Indian:
    freqFilesID.append(freqFilesID_pre.findall(file))

print(freqFilesID)

[['chr1'], ['chr2'], ['chr3'], ['chr4'], ['chr5'], ['chr6'], ['chr7'], ['chr8'], ['chr9'], ['chr10'], ['chr11'], ['chr12'], ['chr13'], ['chr14'], ['chr15'], ['chr16'], ['chr17'], ['chr18'], ['chr19'], ['chr20'], ['chr21'], ['chr22']]


In [24]:
cleardb('dbase_Sqlite', 'Malay_rsID')

## > Processing the Malay vcf to generate rsID table
### >>> created only for the Singapore Malay vcf as command below produced error as such:: 
```
    $ bcftools query -f '%CHROM\t%POS\t%ID\n' SSM.chr8.2012_05.genotypes.vcf.gz -o chr8_rsID 
    $ [E::bcf_hdr_add_sample] Empty sample name: trailing spaces/tabs in the header line?
```

In [21]:
# to gunzip vcf.gz for Malay only
startTime = time.time()
for ID in freqFilesID:
    #print('gunzip '+  workingFolder_Malay + '/SSM.' + ID[0] + '.2012_05.genotypes.vcf.gz')
    try:
        proc1 = subprocess.Popen(['gunzip', workingFolder_Malay + '/SSM.' + ID[0] + '.2012_05.genotypes.vcf.gz'],
                             stdin=subprocess.PIPE, stdout=subprocess.PIPE)
        out, err = proc1.communicate()
        logging.info('gunzip ' +  workingFolder_Malay + '/SSM.' + ID[0] + '.2012_05.genotypes.vcf.gz')
    except:
        logging.info(workingFolder_Malay + '/SSM.' + ID[0] + '.2012_05.genotypes.vcf.gz not present')
        pass
timeTaken = time.time()-startTime 
print(datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S'), 
      '- gunzip completed: Took {} seconds to complete.'.format(timeTaken))

2017/09/13 11:03:08 - gunzip completed: Took 0.13221001625061035 seconds to complete.


In [ ]:
# Manually load data from a raw vcf for Malay population only
startTime = time.time()
for ID in freqFilesID:    
    loaddb_vcf_rsID('Malay_rsID', 'dbase_Sqlite', workingFolder_Malay + '/SSM.' + ID[0] + '.2012_05.genotypes.vcf')
    logging.info('Inserting values from ' +  workingFolder_Malay + 'SSM.' + ID[0] + 
                 '.2012_05.genotypes.vcf to Malay_rsID table of dbase_Sqlite database')
timeTaken = time.time()-startTime 
print(datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S'), 
      '- Data Loading for Malay_rsID of dbase_Sqlite database completed: Took {} seconds to complete.'.format(timeTaken))

1083000 rows loaded
1133594 rows loaded
968661 rows loaded
971842 rows loaded
867221 rows loaded


## > Processing the rest of the tables

In [ ]:
# For Malay Data
startTime = time.time()

for ID in freqFilesID:
    loaddb('Malay_Data', 'dbase_Sqlite', workingFolder_Malay + ID[0] +'_analysis_exome.frq')
    logging.info('Inserting values from ' +  workingFolder_Malay + ID[0] + '_analysis_exome.frq '
                 'to Malay_Data table of dbase_Sqlite database')
timeTaken = time.time()-startTime 
print(datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S'), 
      '- Data Loading for Malay_Data table of dbase_Sqlite database completed: Took {} '
      'seconds to complete.'.format(timeTaken))

In [ ]:
# For Indian Data and rsID
startTime = time.time()

for ID in freqFilesID:
    loaddb('Indian_Data', 'dbase_Sqlite', workingFolder_Indian + ID[0] +'_analysis_exome.frq')
    logging.info('Inserting values from ' +  workingFolder_Indian + ID[0] + '_analysis_exome.frq '
                 'to Indian_Data table of dbase_Sqlite database')
timeTaken = time.time()-startTime 
print(datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S'), 
      '- Data Loading for Indian_Data table of dbase_Sqlite database completed: Took {} '
      'seconds to complete.'.format(timeTaken))

for ID in freqFilesID:
    loaddb('Indian_rsID', 'dbase_Sqlite', workingFolder_Indian + ID[0] +'_rsID')
    logging.info('Inserting values from ' +  workingFolder_Indian + ID[0] + '_analysis_exome.frq '
                 'to Indian_rsID table of dbase_Sqlite database')
timeTaken = time.time()-startTime 
print(datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S'), 
      '- Data Loading for Indian_rsID table of dbase_Sqlite database completed: Took {} '
      'seconds to complete.'.format(timeTaken))

In [ ]:
# For Chinese Data and rsID
startTime = time.time()

for ID in freqFilesID:
    loaddb('Chinese_Data', 'dbase_Sqlite', workingFolder_Chinese + ID[0] +'_analysis_exome.frq')
    logging.info('Inserting values from ' +  workingFolder_Chinese + ID[0] + '_analysis_exome.frq '
                 'to Chinese_Data table of dbase_Sqlite database')
timeTaken = time.time()-startTime 
print(datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S'), 
      '- Data Loading for Chinese_Data table of dbase_Sqlite database completed: Took {} '
      'seconds to complete.'.format(timeTaken))

for ID in freqFilesID:
    loaddb('Chinese_rsID', 'dbase_Sqlite', workingFolder_Chinese + ID[0] +'_rsID')
    logging.info('Inserting values from ' +  workingFolder_Chinese + ID[0] + '_analysis_exome.frq '
                 'to Chinese_rsID table of dbase_Sqlite database')
timeTaken = time.time()-startTime 
print(datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S'), 
      '- Data Loading for Chinese_rsID table of dbase_Sqlite database completed: Took {} '
      'seconds to complete.'.format(timeTaken))

In [ ]:
conn = sqlite3.connect('dbase_Sqlite')
df = pd.read_sql_query("select * from Malay_rsID limit 12", conn)
df

In [ ]:
# Obtain dataset

# for ID in freqFilesID:
#     loaddb('Indian', 'dbase_Sqlite', workingFolder_Indian + ID)
#     df2 = csv.read(workingFolder_Indian + ID[0] + "_rsID", header=False, schema=schema_rsID, sep='\t').alias('df2')
#     freqChrN_working = df2.join(df1, df2.POS == df1.POS).select('df1.*','df2.ID')
#     freqDF_Indian = freqDF_Indian.union(freqChrN_working)
    
# for ID in freqFilesID:
#     df1 = csv.read(workingFolder_Malay + ID[0] + "_analysis_exome.frq", header=True, schema=schema_Freq, sep='\t').alias('df1')
#     df2 = os.read.csv(workingFolder_Malay + ID[0] + "_rsID", header=False, schema=schema_rsID, sep='\t').alias('df2')
#     freqChrN_working = df2.join(df1, df2.POS == df1.POS).select('df1.*','df2.ID')
#     freqDF_Malay = freqDF_Malay.union(freqChrN_working)

# for ID in freqFilesID:
#     df1 = os.read.csv(workingFolder_Chinese + ID[0] + "_analysis_exome.frq", header=True, schema=schema_Freq, sep='\t').alias('df1')
#     df2 = os.read.csv(workingFolder_Chinese + ID[0] + "_rsID", header=False, schema=schema_rsID, sep='\t').alias('df2')
#     freqChrN_working = df2.join(df1, df2.POS == df1.POS).select('df1.*','df2.ID')
#     freqDF_Chinese = freqDF_Chinese.union(freqChrN_working) 